# Calculate global-mean surface temperature and ITCZ positon for TRACMIP aquaplanet runs

 * Global-mean time-mean surface temperature is calculated for aquaControl and aqua4xCO2 runs, using the last 20 years.
 * ITCZ is calculated from time-mean zonal-mean precipitation, again using the last 20 years.
 * For CALTECH model for idealized radiation, the aqua4xCO2 analogue is aquaAbs15.
 * The calculations correspond to Tab. 4 of Voigt et al., 2016, JAMES, https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2016MS000748 .

## Load external libraries

In [1]:
import numpy as np
from intake import open_catalog

## Open Tracmip collection

In [2]:
# get whole pangeo catalogue
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/climate.yaml")
# select tracmip collection
col = cat.tracmip()

## Load aquaplanet monthly data for ts (surface temperature) and pr (precipitation) into a dataset dictionary

In [4]:
ds_dict = col.search(frequency='Amon', experiment=['aquaControl', 'aqua4xCO2', 'aquaAbs15'],
                     variable=['ts','pr']).to_dataset_dict(zarr_kwargs={'consolidated': True})

Progress: |███████████████████████████████████████████████████████████████████████████████| 100.0% 

--> The keys in the returned dictionary of datasets are constructed as follows:
	'model.experiment.frequency'
             
--> There are 28 group(s)


In [5]:
# uncomment the following line to check content of dataset dictionary ds_dict
# ds_dict.keys()

In [6]:
# lists of models
models = list(col.df.model.unique())

## Calculate global-mean surface temperature for each model and experiment, and print to screen

These values can be compared to Tab. 4 of Voigt et al., 2016, where the aqua4xCO2 values are given as the difference to the aquaControl values.

In [7]:
# helper function to calculate global mean
def get_tsmean(data):
    # time-mean and zonal-mean first
    data_tmzm = data.mean(['time','lon'])
    # mean over latitude, using xarray weighted mean functionality
    # note: need at least xarray 0.15.1, 0.15.0 throws weighted attribute error
    weights = np.cos(np.deg2rad(data_tmzm.lat))
    weights.name = "weights"
    return data_tmzm.weighted(weights).mean('lat').values
    
# calculate global-mean surface temperature, only use last 20 years, and print to screen
for mod in models:
    ts_aqctl = get_tsmean(ds_dict[mod+'.aquaControl.Amon']['ts'][-240:,:,:])
    for exp in ['aqua4xCO2', 'aquaAbs15']: # for CALTECH, aqua4xCO2 is given by aquaAbs15
        if mod+'.'+exp+'.Amon' in ds_dict.keys(): # checks that data for either aqua4xCO2 or aquaAbs15 is available
            ts_aq4x = get_tsmean(ds_dict[mod+'.'+exp+'.Amon']['ts'][-240:,:,:])
    # print to screen: aquaControl value and response to 4xCO2
    print(mod, 'aquaControl:'      , np.round(ts_aqctl, decimals=1), 
               'response to 4xCO2:', np.round(ts_aq4x-ts_aqctl, decimals=1))

AM21 aquaControl: 300.7 response to 4xCO2: 6.7
CAM4 aquaControl: 295.1 response to 4xCO2: 4.7
CAM5Nor aquaControl: 290.4 response to 4xCO2: 3.1
CNRM-AM5 aquaControl: 295.2 response to 4xCO2: 6.7
ECHAM61 aquaControl: 295.3 response to 4xCO2: 9.6
ECHAM63 aquaControl: 293.7 response to 4xCO2: 7.8
GISS-ModelE2 aquaControl: 295.0 response to 4xCO2: 4.9
MIROC5 aquaControl: 291.4 response to 4xCO2: 3.8
MPAS aquaControl: 296.8 response to 4xCO2: 2.9
MetUM-CTL aquaControl: 297.2 response to 4xCO2: 7.6
MetUM-ENT aquaControl: 295.6 response to 4xCO2: 7.2
CAM3 aquaControl: 293.8 response to 4xCO2: 3.7
LMDZ5A aquaControl: 292.1 response to 4xCO2: 7.0
CALTECH aquaControl: 291.3 response to 4xCO2: 6.5


## Now also calculate ITCZ position

Again, these values can be compared to Tab. 4 of Voigt et al., 2016, where the aqua4xCO2 values are given as the difference to the aquaControl values.

Note: the ITCZ is defined as the precipitation centroid between 30S/N, where the centroid is understood as the latitude for which 
the same amount of area-integrated precipitation falls north and south of it. Other studies have used the centroid as a precipitaiton- and area-weighted mean of latitude (e.g., Eq.1a of Adam et al., 2016, J. Climate, doi: 10.1175/JCLI-D-15-0512.1) and have also taken other latitudional boundaries (e.g., 20S/N). This might impact the diagnosed ITCZ positions.

In [8]:
# helper function to calculate ITCZ
def get_itcz(pr, lat):
    # input
    # pr: time-mean zonal-mean precipitation
    # lat: latitude 
    latboundary = 30.0 # latitude boundaries for region in which precip is considered
    dlat = 0.01        # latitude spacing of finer grid 
    # make sure that lat increases from SP to NP
    if lat[0]>lat[1]:
        lat = lat[::-1]
        pr  = pr [::-1]
    # interpolate lat and pr on finer dlat grid
    lati  = np.arange(-latboundary, latboundary, dlat)
    pri   = np.interp(lati, lat, pr)
    areai = np.cos(lati*np.pi/180)
    # area-integrated precip (up to constant factor)
    tot = np.sum(pri*areai)
    # integrated pri from -latboundary to lati
    pri_int = np.zeros(lati.size) + np.nan
    for j in range(0, lati.size):
        pri_int[j] = np.sum(pri[0:j+1]*areai[0:j+1])
    itcz = lati[np.argmin(np.abs(pri_int - 0.5*tot))]
    return itcz

# calculate time-mean, zonal-mean ITCZ, only use last 20 years, and print to screen
for mod in models:
    itcz_aqctl = get_itcz(ds_dict[mod+'.aquaControl.Amon']['pr'][-240:,:,:].mean(['time','lon']).values, 
                          ds_dict[mod+'.aquaControl.Amon']['pr'].lat.values)
    for exp in ['aqua4xCO2', 'aquaAbs15']: # for CALTECH, aqua4xCO2 is given by aquaAbs15
        if mod+'.'+exp+'.Amon' in ds_dict.keys(): # checks that data for either aqua4xCO2 or aquaAbs15 is available
            itcz_aq4x = get_itcz(ds_dict[mod+'.'+exp+'.Amon']['pr'][-240:,:,:].mean(['time','lon']).values,
                                 ds_dict[mod+'.aquaControl.Amon']['pr'].lat.values)
    # print to screen: aquaControl value and response to 4xCO2
    print(mod, 'aquaControl:'      , np.round(itcz_aqctl, decimals=1), 
               'response to 4xCO2:', np.round(itcz_aq4x-itcz_aqctl, decimals=1))

AM21 aquaControl: 10.7 response to 4xCO2: -0.4
CAM4 aquaControl: 2.8 response to 4xCO2: 1.1
CAM5Nor aquaControl: 4.0 response to 4xCO2: 0.4
CNRM-AM5 aquaControl: 1.8 response to 4xCO2: 2.3
ECHAM61 aquaControl: 4.5 response to 4xCO2: 7.7
ECHAM63 aquaControl: 3.6 response to 4xCO2: 1.5
GISS-ModelE2 aquaControl: 2.6 response to 4xCO2: 2.3
MIROC5 aquaControl: 2.0 response to 4xCO2: 0.1
MPAS aquaControl: 2.5 response to 4xCO2: 1.0
MetUM-CTL aquaControl: 5.6 response to 4xCO2: 5.5
MetUM-ENT aquaControl: 5.2 response to 4xCO2: 4.1
CAM3 aquaControl: 2.0 response to 4xCO2: -0.3
LMDZ5A aquaControl: 4.3 response to 4xCO2: 4.0
CALTECH aquaControl: 0.8 response to 4xCO2: -0.0
